In [420]:
#imports
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from nba_api.stats.endpoints import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import time
from datetime import datetime, timedelta
from nba_api.stats.library.http import NBAStatsHTTP
import joblib

# Set the timeout
NBAStatsHTTP.timeout = 120  # Set to 30 seconds; adjust as needed

In [263]:
def player_name(player_ID):
    players = playerindex.PlayerIndex()
    players_data = players.get_data_frames()[0]
    last_name = players_data.loc[players_data['PERSON_ID'] == player_ID, 'PLAYER_LAST_NAME'].iloc[0]
    first_name = players_data.loc[players_data['PERSON_ID'] == player_ID, 'PLAYER_FIRST_NAME'].iloc[0]
    player_Name = first_name + " " + last_name
    return player_Name
# print(player_name(1629008))
def player_id(player_Name):
    players = playerindex.PlayerIndex()
    name = player_Name.split()
    first_name = name[0]
    last_name = " ".join(name[1:])
    players_data = players.get_data_frames()[0]
    player_ID = players_data.loc[(players_data['PLAYER_FIRST_NAME'] == first_name) & (players_data['PLAYER_LAST_NAME'] == last_name), 'PERSON_ID'].iloc[0]
    return player_ID
# Example, should return 1629008
# player_id('Michael Porter Jr.')

In [78]:
def team_id(team_abbrev):
    teams = teamgamelogs.TeamGameLogs()
    teams_data = teams.get_data_frames()[0]
    team_id = teams_data.loc[teams_data['TEAM_ABBREVIATION'] == team_abbrev, 'TEAM_ID'].iloc[0]
    return team_id
# Example usage, should return: 1610612747
# print(team_id('LAL'))
def team_abbrev(team_ID):
    teams = teamdetails.TeamDetails(team_id= team_ID)
    teams_data = teams.get_data_frames()[0]
    team_abbrev = teams_data.loc[teams_data['TEAM_ID'] == team_ID, 'ABBREVIATION'].iloc[0]
    return team_abbrev
# team_abbrev(1610612747)

In [262]:
#function to get player gamelog based on Player Name rather than ID

def player_gamelog(player_name):
    id = player_id(player_name)
    gamelog = playergamelog.PlayerGameLog(player_id= id)
    exc = gamelog.get_data_frames()[0]
    return gamelog.get_data_frames()[0]
# player_gamelog('Michael Porter Jr.')


In [80]:
def playerVplayer(player_name, vs_player_name):
    id = player_id(player_name)
    vid = player_id(vs_player_name)
    gamelog = playervsplayer.PlayerVsPlayer(vs_player_id=vid, player_id=id)
    return gamelog.get_data_frames()

# playerVplayer('Nikola Jokic', 'Rudy Gobert')

In [264]:
def getWins(team_id):
    teamlog = teamgamelog.TeamGameLog(team_id= team_id)
    df = teamlog.get_data_frames()[0]
    return df['W'][0]
# getWins(1610612742)

def getLosses(team_id):
    teamlog = teamgamelog.TeamGameLog(team_id= team_id)
    df = teamlog.get_data_frames()[0]
    return df['L'][0]
# getLosses(1610612742)

def getL5Wins(team_id, game_date):
    teamlog = teamgamelog.TeamGameLog(team_id= team_id)
    df = teamlog.get_data_frames()[0]
    # Find the index of the row corresponding to the given game_date
    game_idx = df[df['GAME_DATE'] == game_date].index[0]

    # Select the next 5 rows starting from the game_idx
    previous_games = df.iloc[game_idx+1 :game_idx + 6]

    # Count the number of wins ('W') in the selected rows
    num_wins = previous_games['WL'].value_counts().get('W', 0)

    return num_wins
# getL5Wins(1610612742, 'JAN 14, 2025')

def getL5PPG(team_id, game_date):
    teamlog = teamgamelog.TeamGameLog(team_id= team_id)
    df = teamlog.get_data_frames()[0]
    # Find the index of the row corresponding to the given game_date
    game_idx = df[df['GAME_DATE'] == game_date].index[0]

    # Select the next 5 rows starting from the game_idx
    previous_games = df.iloc[game_idx+1 :game_idx + 6]

    # Count the number of wins ('W') in the selected rows
    last5_pts = previous_games['PTS'].sum()
    return last5_pts / 5
# getL5PPG(1610612742, 'JAN 14, 2025')

In [82]:
def age(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    players_data.to_excel('Players_Data.xlsx')
    player_age = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'AGE'].iloc[0]
    return player_age
# age(player_id('Luka Dončić'))

In [83]:
def height(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_height = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'PLAYER_HEIGHT_INCHES'].iloc[0]
    return player_height
# Example, should return 83(in)
# height(player_id('Nikola Jokić'))

In [84]:
def weight(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_weight = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'PLAYER_WEIGHT'].iloc[0]
    return player_weight
# weight(player_id('Luka Dončić'))

In [266]:
def encode_positions(position):
    roles = {'Primary_Guard': 0, 'Primary_Forward': 0, 'Primary_Center': 0, 'Is_Guard': 0, 'Is_Forward': 0, 'Is_Center': 0}
    primary = position.split('-')[0]
    if 'G' == primary:
        roles['Primary_Guard'] = 1
    elif 'F' == primary:
        roles['Primary_Forward'] = 1
    elif 'C' == primary:
        roles['Primary_Center'] = 1
    if 'G' in position:
        roles['Is_Guard'] = 1
    if 'F' in position:
        roles['Is_Forward'] = 1
    if 'C' in position:
        roles['Is_Center'] = 1
    return pd.Series(roles)
# print(encode_positions('C-G'))
def position(player_ID):
    player_Name = player_name(player_ID)
    name = player_Name.split()
    first_name = name[0]
    last_name = " ".join(name[1:])
    players = playerindex.PlayerIndex()
    df = players.get_data_frames()[0]
    position = df.loc[(df['PLAYER_LAST_NAME'] == last_name) & (df['PLAYER_FIRST_NAME'] == first_name), 'POSITION'].iloc[0]
    return position 
position(1629008) 
# "LeBron, F"

'F'

In [435]:
def player_team(player_name):
    name = player_name.split()
    first_name = name[0]
    last_name = " ".join(name[1:])
    players = playerindex.PlayerIndex()
    df = players.get_data_frames()[0]
    df.to_excel('Player_Index.xlsx')
    team_ID = df.loc[(df['PLAYER_LAST_NAME'] == last_name) & (df['PLAYER_FIRST_NAME'] == first_name), 'TEAM_ID'].iloc[0]
    team = team_abbrev(team_ID)
    return team
player_team('Collin Sexton')

'UTA'

In [87]:
# Will add a Home_Away column to the gamelog so that the ML model can use that as a variable
def homeOrAway(player_gamelog):
    player_gamelog['Home_Away'] = player_gamelog['MATCHUP'].apply(lambda x: 0 if '@' in x else 1)
    return player_gamelog
# kingStats = player_gamelog('LeBron James')
# homeOrAway

In [88]:
def extract_team(matchup):
     # The opponent's team abbreviation is after "vs. " or "@ "
    if " vs. " in matchup:
        # Split the string at " vs. " and take the second part
        return matchup.split(" vs. ")[0]
    elif " @ " in matchup:
        # Split the string at " @ " and take the second part
        return matchup.split(" @ ")[0]
    else:
        return "Invalid Matchup Format"
# print(extract_team('LAL @ NYK'))

def extract_opponent(matchup):
    # The opponent's team abbreviation is after "vs. " or "@ "
    if " vs. " in matchup:
        # Split the string at " vs. " and take the second part
        return matchup.split(" vs. ")[1]
    elif " @ " in matchup:
        # Split the string at " @ " and take the second part
        return matchup.split(" @ ")[1]
    else:
        return "Invalid Matchup Format"
# extract_opponent('LAL @ NYK')

In [89]:
def againstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev):
    awaySearchString = home_team_abbrev + ' vs. ' + opp_team_abbrev
    homeSearchString = home_team_abbrev + ' @ ' + opp_team_abbrev
    newGameLog = player_gamelog.loc[player_gamelog.MATCHUP.isin([homeSearchString, awaySearchString])]
    return newGameLog

# Test
# kingStats = player_gamelog('LeBron James')
# print(againstThisTeam(kingStats, 'LAL', 'ATL'))
def boxStatsAgainstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev):
    new_log = againstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev)
    new_log = new_log.loc[:, ['GAME_DATE', 'MATCHUP', 'REB', 'AST', 'STL', 'BLK', 'PTS', 'MIN']]
    return new_log
# test
# dfoxStats = player_gamelog("De'Aaron Fox")
# boxStatsAgainstThisTeam(dfoxStats, 'SAC', 'MEM')

In [90]:
def getOFFefficiency(team_ID):
    offenses = teamestimatedmetrics.TeamEstimatedMetrics()
    df = offenses.get_data_frames()[0]
    defEfficiency = df.loc[df['TEAM_ID'] == team_ID, 'E_OFF_RATING'].iloc[0]
    return defEfficiency
# print(getOFFefficiency(team_id('CLE')))
def getPPG(team_ID):
    team_stats = leaguestandingsv3.LeagueStandingsV3()
    df_teamstats = team_stats.get_data_frames()[0]
    team_row = df_teamstats[df_teamstats['TeamID'] == team_ID]
    if not team_row.empty:
        totalPTSA = team_row.iloc[0]['PointsPG']
        return totalPTSA
    else:
        print("No data found for the specified city")
        return None
# getPPG(team_id('LAL'))

In [91]:
# Note: the numbers in the def efficiency do not match the numbers on the official website, yet they are in the correct rankings, (i.e OKC's def efficiency is #1 in this data, and #1 on the official data, however they have differing def efficiency numbers, one could be 102.9 the other 103.1)
def getDEFefficiency(team_ID):
    defenses = teamestimatedmetrics.TeamEstimatedMetrics()
    df = defenses.get_data_frames()[0]
    defEfficiency = df.loc[df['TEAM_ID'] == team_ID, 'E_DEF_RATING'].iloc[0]
    return defEfficiency
# print(getDEFefficiency(team_id('LAL')))
    
def getPPG_allowed(team_ID):
    team_stats = leaguestandingsv3.LeagueStandingsV3()
    df_teamstats = team_stats.get_data_frames()[0]
    team_row = df_teamstats[df_teamstats['TeamID'] == team_ID]
    if not team_row.empty:
        totalPTSA = team_row.iloc[0]['OppPointsPG']
        return totalPTSA
    else:
        print("No data found for the specified city")
        return None
# Example Usage, Output: 113.70588235294117
# getPPG_allowed(team_id('LAL'))


In [458]:
playerList = [
        "Alperen Sengun", "Anfernee Simons", "Anthony Davis", "Anthony Edwards", "Austin Reaves", "Brandon Miller", "Bradley Beal", "Cade Cunningham", "Cameron Johnson", "Coby White", "Collin Sexton", "Damian Lillard", "Darius Garland", "De'Aaron Fox", "DeMar DeRozan", "Desmond Bane", "Devin Booker", "Domantas Sabonis", "Donovan Mitchell", "Evan Mobley", "Giannis Antetokounmpo", "Jalen Brunson", "Jalen Green", "Jalen Johnson", "Jalen Williams", "Jamal Murray", "James Harden", "Jaren Jackson Jr.", "Jaylen Brown", "Jayson Tatum", "John Collins", "Jordan Poole", "Julius Randle", "Karl-Anthony Towns", "Kevin Durant", "Kristaps Porziņģis", "Kyrie Irving", "LaMelo Ball", "Lauri Markkanen", "LeBron James", "Malik Monk", "Michael Porter Jr.", "Mikal Bridges", "Nikola Jokić", "Nikola Vučević", "Norman Powell", "Pascal Siakam", "RJ Barrett", "Shaedon Sharpe", "Shai Gilgeous-Alexander", "Stephen Curry", "Trae Young", "Tyler Herro", "Tyrese Haliburton", "Tyrese Maxey", "Victor Wembanyama", "Zach LaVine"
    ]
def createPlayerTeamDict():
    playerTeamDict = {}
    for player in playerList:
        team = player_team(player)  # Get the team for this player
        playerTeamDict[player] = team
    return playerTeamDict
# createPlayerTeamDict()
playerTeamDict = {'Alperen Sengun': 'HOU', 'Anfernee Simons': 'POR', 'Anthony Davis': 'LAL', 'Anthony Edwards': 'MIN', 'Austin Reaves': 'LAL', 'Brandon Miller': 'CHA', 'Bradley Beal': 'PHX', 'Cade Cunningham': 'DET', 'Cameron Johnson': 'BKN', 'Coby White': 'CHI', 'Collin Sexton': 'UTA', 'Damian Lillard': 'MIL', 'Darius Garland': 'CLE', "De'Aaron Fox": 'SAC', 'DeMar DeRozan': 'SAC', 'Desmond Bane': 'MEM', 'Devin Booker': 'PHX', 'Domantas Sabonis': 'SAC', 'Donovan Mitchell': 'CLE', 'Evan Mobley': 'CLE', 'Giannis Antetokounmpo': 'MIL', 'Jalen Brunson': 'NYK', 'Jalen Green': 'HOU', 'Jalen Johnson': 'ATL', 'Jalen Williams': 'OKC', 'Jamal Murray': 'DEN', 'James Harden': 'LAC', 'Jaren Jackson Jr.': 'MEM', 'Jaylen Brown': 'BOS', 'Jayson Tatum': 'BOS', 'John Collins': 'UTA', 'Jordan Poole': 'WAS', 'Julius Randle': 'MIN', 'Karl-Anthony Towns': 'NYK', 'Kevin Durant': 'PHX', 'Kristaps Porziņģis': 'BOS', 'Kyrie Irving': 'DAL', 'LaMelo Ball': 'CHA', 'Lauri Markkanen': 'UTA', 'LeBron James': 'LAL', 'Malik Monk': 'SAC', 'Michael Porter Jr.': 'DEN', 'Mikal Bridges': 'NYK', 'Nikola Jokić': 'DEN', 'Nikola Vučević': 'CHI', 'Norman Powell': 'LAC', 'Pascal Siakam': 'IND', 'RJ Barrett': 'TOR', 'Shaedon Sharpe': 'POR', 'Shai Gilgeous-Alexander': 'OKC', 'Stephen Curry': 'GSW', 'Trae Young': 'ATL', 'Tyler Herro': 'MIA', 'Tyrese Haliburton': 'IND', 'Tyrese Maxey': 'PHI', 'Victor Wembanyama': 'SAS', 'Zach LaVine': 'CHI'}
def playedYesterday():
    tlog = leaguegamelog.LeagueGameLog()
    df = tlog.get_data_frames()[0]
    yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    yesterdayGames = df[df['GAME_DATE'] == yesterday]
    yesterdayTeams = yesterdayGames['TEAM_ABBREVIATION'].tolist()
    
    played = []
    for player in playerList:
        team = playerTeamDict.get(player)  # Get the team of the player
        if team in yesterdayTeams:
            played.append(player)
    
    return played
# playedYesterday()

In [97]:
def concatAllCSVS(folder):
    # List to hold dataframes
    dfs = []
    
    # Iterate over each file in the directory
    for filename in os.listdir(folder):
        if filename.endswith('.csv'):
            # Construct full file path
            file_path = os.path.join(folder, filename)
            # Read the CSV file and append to list
            df = pd.read_csv(file_path, index_col=0)
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            df = df.iloc[7:].reset_index(drop=True)
            average_points = df['PTS'].mean()
            df['target'] = (df['PTS'] > average_points).astype(int)
            df.dropna(inplace=True)
            dfs.append(df)
    
    # Concatenate all dataframes in the list
    full_df = pd.concat(dfs, ignore_index=True)
    return full_df
# master_csv = concatAllCSVS('players csvs')

In [223]:
# THIS IS TO CREATE A MODEL WITH EVERYONE'S CSVS CONCATTED
def createModelALL():
    file_path = 'master v2.1.csv'  # Specify your CSV file name here
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)
    # Calculate average points
    average_points = df['PTS'].mean()
    df['target'] = (df['PTS'] > average_points).astype(int)

    # Features including PLAYER_ID
    categorical_features = ['MATCHUP', 'Player_ID']
    classifier = Pipeline([
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
            remainder='passthrough')),
        ('classifier', LogisticRegression(solver = 'saga', max_iter=50000, warm_start=True))
    ])
    # 
    X = df[['MATCHUP', 'Player_ID', 'MA_PTS', 'MA_FGM', 'MA_FGA', 'MA_STL', 'MA_MIN', 'MA_PLUS_MINUS', 'AVG_PTS', 'MA_FG_PCT', 'OPP_PPG_ALLOWED', 'OPP_DEF_EFFICIENCY', 'PPG', 'OFF_EFFICIENCY',  'Home_Away', 'HEIGHT', 'WEIGHT', 'AGE', 'MA_TOV', 'MA_BLK', 'Primary_Guard', 'Primary_Forward','Primary_Center' , 'Is_Guard', 'Is_Forward', 'Is_Center', 'TEAM_W', 'TEAM_L', 'L5_W', 'L5_L', 'L5_PPG', 'OPP_TEAM_W', 'OPP_TEAM_L', 'OPP_L5_W', 'OPP_L5_L', 'OPP_L5_PPG']]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    # Save the model
    joblib.dump(classifier, 'master_model_libV2.1saga.pkl')
createModelALL()

Accuracy: 0.712280701754386
              precision    recall  f1-score   support

           0       0.74      0.76      0.75       161
           1       0.68      0.65      0.66       124

    accuracy                           0.71       285
   macro avg       0.71      0.70      0.71       285
weighted avg       0.71      0.71      0.71       285



In [235]:
def testNewAccuracy():
    df = pd.read_csv('master v2.1.csv')

    # Assume the data includes a 'target' column and other feature columns
    X = df[['Player_ID', 'MATCHUP', 'MA_PTS', 'MA_FGM', 'MA_FGA', 'MA_STL', 'MA_MIN', 'AVG_PTS', 'MA_FG_PCT', 'MA_PLUS_MINUS', 'OPP_PPG_ALLOWED', 'OPP_DEF_EFFICIENCY', 'PPG', 'OFF_EFFICIENCY', 'HEIGHT', 'WEIGHT', 'AGE', 'MA_TOV', 'MA_BLK', 'Home_Away', 'Primary_Guard', 'Primary_Forward', 'Primary_Center' , 'Is_Guard', 'Is_Forward', 'Is_Center', 'TEAM_W', 'TEAM_L', 'L5_W', 'L5_L', 'L5_PPG', 'OPP_TEAM_W', 'OPP_TEAM_L', 'OPP_L5_W', 'OPP_L5_L', 'OPP_L5_PPG']]
    y = df['target']
    #
    # Predict using the model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Load the trained model
    model = joblib.load('master_model_libV2.1.pkl')

    # Evaluate the model on the new data
    predictions = model.predict(X_test)
    print("Accuracy on updated data:", accuracy_score(y_test, predictions))

testNewAccuracy()

Accuracy on updated data: 0.6421052631578947


In [459]:
# NOTE!!!!!!!!!!!! THIS SHOULD ONLY BE USED ONCE TO CREATE A NEW CSV FOR A PLAYER. ONCE A CSV HAS BEEN CREATED, USE THE updateStats METHOD TO UPDATE THEIR MOST CURRENT GAME.

def importantStatsRev(player_name):
    player_gameLog = player_gamelog(player_name)
    player_gameLog = player_gameLog.iloc[::-1].reset_index(drop=True)
    player_gameLog = player_gameLog[player_gameLog['MIN'] >= 20]
    window_size = 5
    stats_to_average = ['PTS', 'FGM', 'FGA', 'STL', 'BLK', 'TOV', 'FG_PCT', 'MIN', 'PLUS_MINUS']
    for stat in stats_to_average:
        player_gameLog[f'MA_{stat}'] = player_gameLog[stat].rolling(window=window_size, min_periods=1).mean().shift(0)
    player_gameLog['AVG_PTS'] = player_gameLog['PTS'].expanding().mean().shift(1)

    # Extract opponent team abbreviation
    player_gameLog['TEAM_ABBREVIATION'] = player_gameLog['MATCHUP'].apply(extract_team)
    player_gameLog['TEAM_ID'] = player_gameLog['TEAM_ABBREVIATION'].apply(team_id)
    player_gameLog['OPP_TEAM'] = player_gameLog['MATCHUP'].apply(extract_opponent)
    player_gameLog['OPP_TEAMID'] = player_gameLog['OPP_TEAM'].apply(team_id)
    # Fetch the PPG allowed by the opponent team
    player_gameLog['OPP_PPG_ALLOWED'] = player_gameLog['OPP_TEAMID'].apply(getPPG_allowed)
    player_gameLog['OPP_DEF_EFFICIENCY'] = player_gameLog['OPP_TEAMID'].apply(getDEFefficiency)
    player_gameLog['PPG'] = player_gameLog['TEAM_ID'].apply(getPPG)
    player_gameLog['OFF_EFFICIENCY'] = player_gameLog['TEAM_ID'].apply(getOFFefficiency)
    player_gameLog['POSITION'] = player_gameLog['Player_ID'].apply(position)
    encoded_positions = player_gameLog['POSITION'].apply(encode_positions)
    player_gameLog = pd.concat([player_gameLog, encoded_positions], axis=1)
    player_gameLog['AGE'] = player_gameLog['Player_ID'].apply(age)
    player_gameLog['HEIGHT'] = player_gameLog['Player_ID'].apply(height)
    player_gameLog['WEIGHT'] = player_gameLog['Player_ID'].apply(weight)
    player_gameLog['TEAM_W'] = player_gameLog['TEAM_ID'].apply(getWins)
    player_gameLog['TEAM_L'] = player_gameLog['TEAM_ID'].apply(getLosses)
    player_gameLog['L5_W'] = player_gameLog.apply(lambda row: getL5Wins(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['L5_L'] = 5 - player_gameLog['L5_W']
    player_gameLog['L5_PPG'] = player_gameLog.apply(lambda row: getL5PPG(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_TEAM_W'] = player_gameLog['OPP_TEAMID'].apply(getWins)
    player_gameLog['OPP_TEAM_L'] = player_gameLog['OPP_TEAMID'].apply(getLosses)
    player_gameLog['OPP_L5_W'] = player_gameLog.apply(lambda row: getL5Wins(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_L5_L'] = 5 - player_gameLog['L5_W']
    player_gameLog['OPP_L5_PPG'] = player_gameLog.apply(lambda row: getL5PPG(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)

    # Reset the index if needed (optional, for cleaner indexing)
    player_gameLog.reset_index(drop=True, inplace=True)
    homeOrAway(player_gameLog)
    return player_gameLog
# Example Usage, runtime ~1m 18s
players_list_all = [
    
]
for player in players_list_all:
    file_path = os.path.join('v2.1 csvs', player + '.csv')
    df_updated = importantStatsRev(player)
    df_updated.to_csv(file_path)
    print("This player's csv was created: ", player)
# masterTESTcsv = concatAllCSVS('v2.1 csvs')
# masterTESTcsv.to_csv('masterCSV_2025-01-20 v2.1 newer.csv')

In [466]:
# USE THIS METHOD ONCE A CSV HAS BEEN CREATED FOR A PLAYER
def updateStatsRev(player_csv, player_name):
    df = pd.read_csv(player_csv)
    df_new = importantStatsRev(player_name)
    last_row = df_new.tail(1)
    df = pd.concat([df, last_row])
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    if df.iloc[-1]['GAME_DATE'] == df.iloc[-2]['GAME_DATE']:
        df = df.iloc[:-1]
        print(f"This {player_name} CSV is up to date")
    else:
        print(f"Updated {player_name} CSV")
        # file_path = os.path.join('test csvs', player_name + '.csv')
        # df.to_csv(file_path, index=False)
    return df

playedList = playedYesterday()

def updateAllRev(player_list, folder):
    for player in player_list:
        file_path = os.path.join(folder, player + '.csv')
        df_updated = updateStatsRev(file_path, player)
        df_updated.to_csv(file_path)
# updateAllRev(playedList, 'v2.1 csvs')
df = concatAllCSVS('v2.1 csvs')
today = datetime.today().strftime('%Y-%m-%d')
file_name = f"masterCSV_{today} v2.1.csv"
df.to_csv(file_name, index=False)

In [468]:
def testNewAccuracy(iterations=500):
    # Load the dataset
    df = pd.read_csv('masterCSV_2025-01-20 v2.1.csv')

    # Define features and target
    X = df[['Player_ID', 'MATCHUP', 'MA_PTS', 'MA_FGM', 'MA_FGA', 'MA_STL', 'MA_MIN', 'AVG_PTS', 'MA_FG_PCT', 'MA_PLUS_MINUS', 
            'OPP_PPG_ALLOWED', 'OPP_DEF_EFFICIENCY', 'PPG', 'OFF_EFFICIENCY', 'HEIGHT', 'WEIGHT', 'AGE', 'MA_TOV', 'MA_BLK', 
            'Home_Away', 'Primary_Guard', 'Primary_Forward', 'Primary_Center', 'Is_Guard', 'Is_Forward', 'Is_Center', 
            'TEAM_W', 'TEAM_L', 'L5_W', 'L5_L', 'L5_PPG', 'OPP_TEAM_W', 'OPP_TEAM_L', 'OPP_L5_W', 'OPP_L5_L', 'OPP_L5_PPG']]
    #, 'OPP_TEAM_W', 'OPP_TEAM_L', 'OPP_L5_W', 'OPP_L5_L', 'OPP_L5_PPG'
    y = df['target']

    # Load the trained model
    model = joblib.load('master_model_libV2.1.pkl')

    # List to store accuracies
    accuracies = []

    # Run the testing process multiple times
    for i in range(iterations):
        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)  # Vary random_state for different splits

        # Predict and calculate accuracy
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        accuracies.append(accuracy)

    # Calculate the average accuracy
    average_accuracy = np.mean(accuracies)
    print(f"\nAverage Accuracy over {iterations} iterations: {average_accuracy:.4f}")

# Run the function with the desired number of iterations
testNewAccuracy(iterations=500)


Average Accuracy over 500 iterations: 0.8154


In [467]:
def warm_start_train(old_model_path, data, output_model_path):
    """
    Train LogisticRegression with warm start using combined old and new data.

    Parameters:
    - old_model_path: Path to the previously saved model.
    - new_data_path: Path to the new data CSV.
    - master_csv_path: Path to the master dataset CSV.
    - output_model_path: Path to save the updated model.
    """
    # Load new data
    new_data = pd.read_csv(data)
    
    # Define features and target
    X = new_data[['Player_ID', 'MATCHUP', 'MA_PTS', 'MA_FGM', 'MA_FGA', 'MA_STL', 'MA_MIN', 'AVG_PTS', 
                       'MA_FG_PCT', 'MA_PLUS_MINUS', 'OPP_PPG_ALLOWED', 'OPP_DEF_EFFICIENCY', 'PPG', 
                       'OFF_EFFICIENCY', 'HEIGHT', 'WEIGHT', 'AGE', 'MA_TOV', 'MA_BLK', 'Home_Away', 
                       'Primary_Guard', 'Primary_Forward', 'Primary_Center', 'Is_Guard', 'Is_Forward', 
                       'Is_Center', 'TEAM_W', 'TEAM_L', 'L5_W', 'L5_L', 'L5_PPG', 'OPP_TEAM_W', 'OPP_TEAM_L', 'OPP_L5_W', 'OPP_L5_L', 'OPP_L5_PPG']]
    y = new_data['target']
    # Train the model on the combined data
    model = joblib.load(old_model_path)
    model.fit(X, y)
    print("Model training completed.")

    # Save the updated model
    joblib.dump(model, output_model_path)
    print(f"Updated model saved to {output_model_path}")

# Example usage
warm_start_train(
    old_model_path='master_model_libV2.1.pkl', 
    data = 'masterCSV_2025-01-20 v2.1.csv',
    output_model_path='master_model_libV2.1.pkl'
)


Model training completed.
Updated model saved to master_model_libV2.1.pkl


In [409]:
def getPredictData(player_name, matchup):
    # Construct the file path using the player's name
    file_path = f'v2.1 csvs/{player_name}.csv' 
    df = pd.read_csv(file_path)
    df = df.tail(1)
    if ' @ ' in matchup:
        teams = matchup.split(' @ ')
        home_away = 0  
    elif ' vs. ' in matchup:
        teams = matchup.split(' vs. ')
        home_away = 1 
    df['MATCHUP'] = matchup
    df['OPP_DEF_EFFICIENCY'] = getDEFefficiency(team_id(teams[1]))
    df['OPP_PPG_ALLOWED'] = getPPG_allowed(team_id(teams[1]))
    df['PPG'] = getPPG(team_id(teams[0]))
    df['OFF_EFFICIENCY'] = getOFFefficiency(team_id(teams[0]))
    df['Home_Away'] = home_away
    return df

def predict_performance(new_data):
    classifier = joblib.load('master_model_libV2.1.pkl')
    
    predictions = classifier.predict(new_data)
    probabilities = classifier.predict_proba(new_data)
    
    return predictions, probabilities

# Example usage
# Assuming 'new_data' is DataFrame with the same features used in training
new_data = getPredictData("Coby White", 'CHI @ LAC')
performance_predictions, performance_probabilities = predict_performance(new_data)
print("Line: " + str(new_data['AVG_PTS'].values[0]) + " PTS")
print("Predictions:", performance_predictions)  
print("Probabilities:", performance_probabilities)

Line: 18.46153846153846 PTS
Predictions: [0]
Probabilities: [[0.74197715 0.25802285]]


In [408]:
def kelly_criterion(bankroll, vegas_odds, probability):
    if vegas_odds > 0:
        b = vegas_odds / 100
    else:
        b = -100 / vegas_odds
    
    # Calculate the Kelly fraction
    q = 1 - probability
    kelly_fraction = (b * probability - q) / b
    
    # Calculate the bet size
    bet_size = kelly_fraction * bankroll
    
    return max(0, bet_size)  # Ensure the bet size is not negative

bankroll = 22  # $1,000 available for betting
vegas_odds = +168 # +150 odds
probability = 0.60426302# 55% chance of winning

bet_amount = kelly_criterion(bankroll, vegas_odds, probability)
print(f"You should bet ${bet_amount:.2f}")

You should bet $8.11
